**DTS - Complete Streets**
# 05 - Modal Widths



**Purpose**

1. Develop individual modal widths per modal type.
2. Summarize modal widths into an "unconstrained" modal width metric. Various other metrics are then calculated using width esitmates and linear line length. (modal area, unconstrained modal area, exceedence area etc.)




**Global Assumptions and Notes**

global assumptions and notes go here detailed assumptions should go inline with specific code segments.
1. Item
2. item
3. item
4. item

**Non-Standard Python Modules utilized**
1. ArcPy (ArcGIS Pro) required
2. Pandas (optional - only used for QC purposess and data exploration)


**Author:** rmangan


In [ ]:
# import modules
import arcpy
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

HPMS = os.path.join(input_gdb_path,"HPMS_RCL_join")

HERE = os.path.join(input_gdb_path,"RCLs_with_NAVOIDs_35buff_10clip")

HERE_orig = os.path.join(input_gdb_path, "hw_linksSPEED")

medians = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\median_classification.gdb\modal_comp_05_median_classification"

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

modal_composite_url = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

modal_composite_url_copy = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

zoning = r"\\dc1vs01\GISProj\B\Base\HI\Honolulu_City_County\zoning\CCH_zoning.gdb\zoning_LUO_20210505"

In [ ]:
#featureset and updte testing against AGOL FS

#generate token
token = gis._con.token

print("Token: {}".format(str(token)))

modal_composite_fs = modal_composite_url_copy + "?token={}".format(token)

print(modal_composite_fs)

featureset = arcpy.FeatureSet()

fs = gis.content.get('6655329b1fb34a91b419b8d002588524')
fs.url

# lyr = arcpy.management.MakeFeatureLayer(modal_composite, "modal_comp_lyr")[0]


featureset.load(modal_composite_fs)

In [ ]:
type(modal_composite_fs)

In [ ]:
fields = ["SEGMENTID","FULLNAME"]

count = 0

with arcpy.da.SearchCursor(modal_composite_fs, fields) as cursor:
    for row in cursor:
        print(row[1])
        count += 1

        
print(count)
        

## Pre-Processing & Clean up

### Drop Unecessary Fields

In [ ]:
#drop unecessary fields (all fields are avaialble in modal_composite_full dataset)

#get all fields and add to list
# field_list = []

# fields = arcpy.ListFields(modal_composite)

# for field in fields:
#     field_list.append(field.name)
    
field_list = [i.name for i in arcpy.ListFields(modal_composite)]

# for i in field_list:
#     print("\"{}\",".format(i))

#fields to keep

keep_fields = [
    "OBJECTID",
    "Shape",
    "SEGMENTID",
    "FULLNAME",
    "NAME",
    "PROPERNAME",
    "TYPE",
    "POSTDIR",
    "CLASS",
    "STREET_CLA",
    "ZIPCODER",
    "ZIPCODEL",
    "TOWNL",
    "TOWNR",
    "NEIGHBORL",
    "NEIGHBORR",
    "ONEWAY",
    "OWNER",
    "NBRightPed",
    "SBLeftPedZ",
    "NBRightP_1",
    "SBLeftPe_1",
    "NBRightP_2",
    "SBLeftPe_2",
    "NBRightP_3",
    "SBLeftPe_3",
    "NBRightP_4",
    "SBLeftPe_4",
    "NBRightP_5",
    "SBLeftPedB",
    "NBRightP_6",
    "SBLeftPe_5",
    "NBRightP_7",
    "SBLeftPe_6",
    "NBRightP_8",
    "SBLeftPe_7",
    "NBRightVis",
    "SBLeftVisu",
    "MidBlkXwkU",
    "MidBlkXwkC",
    "MidBlkXw_1",
    "TraffCalmi",
    "PPN_Final",
    "HPI",
    "PpnNotCity",
    "FID_Ped_Improve_temp",
    "Ped_Improve_ProjectID",
    "Ped_Immprove_Extents",
    "FID_Ped_Add_temp",
    "Ped_Add_Project_ID",
    "Fac_Name_BE",
    "Fac_Desc_BE",
    "Fac_Type_BE",
    "Fac_Name_BP",
    "Fac_Desc_BP",
    "Fac_Name_BR",
    "Fac_Desc_BR",
    "Fac_Type_BR",
    "TCAD_OID_2020",
    "AB_SPEED",
    "BA_SPEED",
    "AB_CAPACIT",
    "BA_CAPACIT",
    "AB_LANE",
    "BA_LANE",
    "AB_CNT",
    "BA_CNT",
    "AB_FACTYPE",
    "BA_FACTYPE",
    "AB_FNCLASS",
    "BA_FNCLASS",
    "TOT_FLOW_D",
    "max_row_2020",
    "min_row_2020",
    "mean_row_2020",
    "f_class",
    "fedaid_id",
    "ln_exist",
    "ln_prop",
    "row_exist",
    "row_prop",
    "TCAD_OID_2040",
    "ID",
    "AB_VOC_MAX",
    "BA_VOC_MAX",
    "FACTYPE",
    "ATYPE",
    "TPN_OID",
    "FULLNAME_1",
    "Tier",
    "NewTier",
    "BE_OID_tmp",
    "BP_OID_tmp",
    "BR_OID_tmp",
    "PED_ADD_OID_tmp",
    "PED_IMPR_OID_tmp",
    "Shape_Length",
    "ln_exist_num",
    "ln_prop_num",
    "row_exist_num",
    "row_prop_num",
    "modal_width_ped_exist",
    "modal_width_ped_prop",
    "modal_width_bike_exist",
    "modal_width_bike_prop",
    "modal_width_auto_exist",
    "modal_width_auto_prop",
    "modal_width_bus_exist",
    "modal_width_bus_prop",
    "modal_width_park_exist",
    "modal_width_park_prop",
    "modal_width_medians",
    "modal_width_max",
    "Fac_Type_BP",
    "lanes_assumed",
    "VOC_max",
    "median_type",
    "median_width",
    "progression",
    "CS_example_num",
    "CS_example_type",
    "CS_type",
    "length_urban_percent",
    "length_urban",
    "length_ag",
    "length_ag_percent",
    "length_ft",
    "mean_row_area",
    "modal_area_max",
    "modal_area_diff",
    "route_id",
    "aadt",
    "funsystem",
    "funsysdesc",
    "factype_1",
    "factypedes",
    "thrulanes",
    "SPEED",
    "SPD_LIMIT",
    "MIN_SPD_LIMIT",
    "MAX_SPD_LIMIT",
    "LandValue",
    "modal_width_ped_const",
    "modal_width_bike_const",
    "modal_width_auto_const",
    "modal_width_bus_const",
    "modal_width_park_const",
    "modal_width_medians_const",
    "modal_width_const",
    "modal_area_const",
    "modal_area_diff_const",
    "bike_parallel_route",
    "lane_count_rev",
    "cs_type_revise",
    "modal_width_auto_rule",
    "modal_width_ped_furn",
    "row_estimate_note",
    "modal_area_auto_E",
    "modal_area_auto_U",
    "modal_area_auto_C",
    "modal_area_ped_E",
    "modal_area_ped_U",
    "modal_area_ped_C",
    "modal_area_bike_E",
    "modal_area_bike_U",
    "modal_area_bike_C"
]



In [ ]:
#get all fields and check against keep list to create drop list
list_fields = arcpy.ListFields(modal_composite)
fields = [i.name for i in list_fields]

drop_list = []
for i in fields:
    if i in keep_fields:
        print("{} field - keeping".format(i))
    else:
        print("{} field - dropping".format(i))
        drop_list.append(i)

#drop fields
print("dropping fields..")
arcpy.DeleteField_management(modal_composite, drop_list)

print("done")
        



### Add Required Fields (needs cleanup)

In [ ]:
#add field for analysis if not already present

#store field, field alias in dict

#widths = SHORT
#areas = FLOAT
#misc = TEXT

new_fields = {"row_estimate_note":      "ROW Estimation Note", #TEXT
              "cs_type_revise":         "Complete Street Type - Re-Evaluate",
              
              "lanes_assumed":     "Lanes (assumed)",
              "lane_count_exist":  "Transit Lanes - Existing",
              "lane_count_prop":   "Transit Lanes - Proposed (Unconstrained)",
              "lane_count_rev":    "Transit Lanes - Proposed (Constrained)",
              "lane_miles_exist":  "Transit Lane Miles - Existing",
              "lane_miles_prop":   "Transit Lane Miles - Proposed (Unconstrained)",
              "lane_miles_const":  "Transit Lane Miles - Proposed (Constrained)",
              "lane_count_note":   "Transit Lanes - Note",    #TEXT
              
              "modal_width_ped_exist":  "Modal Width - Ped (Existing)",
              "modal_width_ped_prop":   "Modal Width - Ped (Unconstrained)",
              "modal_width_ped_const":  "Modal Width - Ped (Constrained)",
              "modal_area_ped_exist":   "Modal Area - Ped (Existing)",
              "modal_area_ped_prop":    "Modal Area - Ped (Unconstrained)",
              "modal_area_ped_const":   "Modal Area - Ped (Constrained)",
              
              "modal_width_bike_exist": "Modal Width - Bike (Existing)",
              "modal_width_bike_prop":  "Modal Width - Bike (Unconstrained)",
              "modal_width_bike_const": "Modal Width - Bike (Constrained)",
              "modal_area_bike_exist":  "Modal Area - Bike (Existing)",
              "modal_area_bike_prop":   "Modal Area - Bike (Unconstrained)",  
              "modal_area_bike_const":  "Modal Area - Bike (Constrained)",  
              "bike_parallel_route":    "Bike - Evaluate Parallel Routes",        
              
              "modal_width_auto_exist": "Modal Width - Auto (Existing)",
              "modal_width_auto_prop":  "Modal Width - Auto (Unconstrained)",
              "modal_width_auto_const": "Modal Width - Auto (Constrained)",
              "modal_area_auto_exist":  "Modal Area - Auto (Existing)",
              "modal_area_auto_prop":   "Modal Area - Auto (Unconstrained)",
              "modal_area_auto_const":  "Modal Area - Auto (Constrained)",
              "modal_width_auto_rule":  "Modal Width - Auto (Logic QC)",   #TEXT
              
              "modal_width_bus_exist":  "Modal Width - Bus (Existing)",
              "modal_width_bus_prop":   "Modal Width - Bus (Proposed)",
              "modal_width_bus_const":  "Modal Width - Bus (Constrained)",
              "modal_area_bus_exist":  "Modal Area - Bus (Existing)",
              "modal_area_bus_prop":   "Modal Area - Bus (Unconstrained)",
              "modal_area_bus_const":  "Modal Area - Bus (Constrained)",      
              
              "modal_width_park_exist": "Modal Width - Parking (Existing)",
              "modal_width_park_prop":  "Modal Width - Parking (Proposed)",
              "modal_width_park_const": "Modal Width - Parking (Constrained)",
              "modal_area_park_exist":  "Modal Area - Parking (Existing)",
              "modal_area_park_prop":   "Modal Area - Parking (Unconstrained)",
              "modal_area_park_const":  "Modal Area - Parking (Constrained)",           
              
              "modal_width_medians_exist":  "Modal Width - Medians (Existing)",
              "modal_width_medians_prop":  "Modal Width - Medians (Proposed)",
              "modal_width_medians_const":  "Modal Width - Medians (Constrained)",
              "modal_area_medians_exist":  "Modal Area - Medians (Existing)",
              "modal_area_medians_prop":   "Modal Area - Medians (Unconstrained)",
              "modal_area_medians_const":  "Modal Area - Medians (Constrained)",
                 
              "length_ft":              "Length (ft.)",
              "mean_row_area":          "ROW Area Estimate (sq. ft.)",
              
              "modal_width_exist":      "Modal Width - Total Existing",
              "modal_width_max":        "Modal Width - Total Unconstrained",
              "modal_width_max_desc":   "Modal Width - Total Unconstrained Description", #TEXT
              "modal_width_const":      "Modal Width - Total Constrained",
              "modal_width_const_desc": "Modal Width - Total Constrained Description",   #TEXT
  
              "modal_area_max":             "Unconstrained Modal Area (sq. ft.)",
              "modal_area_diff_const":      "Modal Area - Total Constrained Exceedence (sq. ft.)",
              "modal_area_const":           "Modal Area - Total Constrained (sq. ft.)"}

#loop through dict and add fields
    
#create list of field names
fields = [i.name for i in arcpy.ListFields(modal_composite)]

for key, value in new_fields.items():
    if key in fields:
        print("Field {} already exists".format(key))
        print("Updating alias to: {}".format(value))
        arcpy.AlterField_management(in_table = modal_composite,field = key, new_field_alias = value)
           
    else:
        if key in ["lane_count_note",
                   "modal_width_auto_rule",
                   "row_estimate_note",
                   "modal_width_max_desc",
                   "modal_width_const_desc"]:
            print("Adding field {0}".format(key))
            arcpy.AddField_management(modal_composite,field_name=key,field_type="TEXT", field_alias = value)
            
        elif key in ["lane_miles_exist",
                    "lane_miles_prop",
                    "lane_miles_const",  
                    "modal_area_ped_exist",
                    "modal_area_ped_prop",
                    "modal_area_ped_const",           
                    "modal_area_bike_exist",
                    "modal_area_bike_prop",  
                    "modal_area_bike_const",  
                    "modal_area_auto_exist",
                    "modal_area_auto_prop",
                    "modal_area_auto_const",      
                    "modal_area_bus_exist",
                    "modal_area_bus_prop",
                    "modal_area_bus_const",                  
                    "modal_area_park_exist",
                    "modal_area_park_prop",
                    "modal_area_park_const",           
                    "modal_area_medians_exist",
                    "modal_area_medians_prop",
                    "modal_area_medians_const",               
                    "length_ft",
                    "mean_row_area",
                    "modal_area_max",
                    "modal_area_diff_const",
                    "modal_area_const"]:
            
            print("Adding field {0}".format(key))
            arcpy.AddField_management(modal_composite,field_name=key,field_type="FLOAT", field_alias = value) 
            
        else:
            print("Adding field {0}".format(key))
            arcpy.AddField_management(modal_composite,field_name=key,field_type="SHORT", field_alias = value)
            
            
print("done")


In [ ]:
#fix bike proposed missing fac_type field - MOVED TO SCRIPT 04

bike_prop_intr = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb\Bike_Proposed_RCL_intersect"

bike_prop_alias = {
    "Fac_Type":"Facility Type (Bike Proposed)"
}

#rename bike proposed field
for key, value in bike_prop_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {} to field {}, field renamed to {}".format(value,key,new_field_name))
    arcpy.AlterField_management(bike_prop_intr,key,new_field_name, value)
    
    
#join Bike Proposed intersect results
print("Joining Bike Proposed fields...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = bike_prop_intr
join_table_field = "SEGMENTID"
join_fields = ["Fac_Type_BP"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")



### Join misc data (HPMS, medians, etc...)

In [ ]:
#join fields from median classification
print("Joining Median fields...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = medians
join_table_field = "SEGMENTID"
join_fields = ["median_type","median_width","progression"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")


In [ ]:
#Join fieds from HMPS
print("Joining HPMS fields...")
join_target = modal_composite_05_2
join_target_field = "SEGMENTID"
join_table = HPMS
join_table_field = "SEGMENTID"
join_fields = ["aadt","route_id","funsystem", "funsysdesc","factype","factypedes","thrulanes"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HPMS Fields complete.")

#Join fields from HERE
print("Joining HERE fields...")
join_target = modal_composite_05_2
join_target_field = "SEGMENTID"
join_table = HERE
join_table_field = "SEGMENTID"
join_fields = ["SPEED","SPD_LIMIT","MIN_SPD_LIMIT", "MAX_SPD_LIMIT"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HERE Fields complete.")
    
print("Done")


In [ ]:
#update field alias for HPMS and HERE fields

update_alias = {
    "aadt":"HPMS - Annual Average Daily Traffic, Two Directions",
    "route_id":"HPMS - Route Number",
    "funsystem":"HPMS - Functional System Code",
    "funsysdesc":"HPMS - Functional System Code Description",
    "factype":"HPMS - Facility Type Code",
    "factypedes":"HPMS - Facility Type Code Description",
    "thrulanes":"HPMS - Number of Through Lanes, Two Directions",
    "SPD_LIMIT":"Speed Limit (HERE)",
    "MIN_SPD_LIMIT":"Speed Limit (HERE - low)",
    "MAX_SPD_LIMIT":"Speed Limit (HERE - high)"
}

#rename bike proposed field
for key, value in update_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {0} to field {1}".format(value,key))
    arcpy.AlterField_management(modal_composite_05_2,key,new_field_alias=value)
    
print("alias assignment done")

In [ ]:
fields = [i.name for i in arcpy.ListFields(modal_composite)]

for i in fields:
    print(i)

In [ ]:
#join lane values from HERE

#dissovle HERE on LinkID, summing Lane Values together

#Create near table between HERE and RCL with 3 closest matches, 20' search distance. 

#Join Street Name & Lane Summary from HERE. Join Street Name & Segment ID from RCL to near table.

#Discard records of mismatched street names. (here was close to an RCL of another street)

#Summary Stats on table, using SegmentID as case field, get Median Lane value of remaining records

#add median lane value to modal composite




#on HERE, dissolve Lanes(sum) (linkID is dissolve field)
print("dissolving Lanes on LinkID...")
here_dissolve = os.path.join(scratch_gdb_path, "here_dissolve")
arcpy.management.Dissolve(HERE_orig, here_dissolve, dissolve_field = "LINK_ID", statistics_fields = [["LANES","SUM"]])

#generate near table between dissolve output & RCL, find 3 closest features, search radius 20'
print("generating near table...")
near = os.path.join(scratch_gdb_path, "near")
arcpy.analysis.GenerateNearTable(here_dissolve, modal_composite,
                                        near, search_radius = 20,
                                        closest = False, closest_count = 3)

#on near table, join field from dissolve output IN_FID = Dissolve OID, add LinkID & Lane Sum
print("joining LinkID & Lane Summary from dissolve output...")
join_target = near
join_target_field = "IN_FID"
join_table = here_dissolve
join_table_field = "OBJECTID"
join_fields = ["LINK_ID","SUM_LANES"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

#on near table, join field from HERE LinkID = LinkID, add ST_NAME
print("joining ST_NAME from HERE...")
join_target = near
join_target_field = "LINK_ID"
join_table = HERE_orig
join_table_field = "LINK_ID"
join_fields = ["ST_NAME"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

#on near table, join field from RCL NearFID = OID, add SEGMENTID, FULLNAME
print("joining SEGMENTID, FULLNAME from RCL...")
join_target = near
join_target_field = "NEAR_FID"
join_table = modal_composite
join_table_field = "OBJECTID"
join_fields = ["SEGMENTID","FULLNAME"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

#on near table, copy to new dataset keeping only rows with street name matches
print("dropping table rows with street name mismatches...")
near_select = os.path.join(scratch_gdb_path, "near_select")
arcpy.conversion.TableToTable(near, scratch_gdb_path,"near_select" , '"ST_NAME" = "FULLNAME"')

#on near select, run summary statistics, Case Field = SegmentID, SUM_Lanes = Median
print("generating summary stats table of SegmentID and median Lane count...")
lane_summary = os.path.join(scratch_gdb_path, "lane_summary")
arcpy.analysis.Statistics(near_select, lane_summary, statistics_fields = [["SUM_LANES", "MEDIAN"]], case_field = "SEGMENTID")

#on modal_composite, join field from near table on SegmentID, keeping lanes
print("joining final lane output to modal_composite...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = lane_summary
join_table_field = "SEGMENTID"
join_fields = ["MEDIAN_SUM_LANES"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

print("complete.")

### Lane Counts - parse & aggregate from multiple sources

In [ ]:
#fill lane_count with values from multiple fields. Will be used as main lane count indicator in projec.
#Unconstrained & Constrained Lane counts will be updated in subsquent functions

#1 - TransCAD
#2 - HPMS
#3 - 1986 ROW table
#4 - DTS data (if time)
#5 - Assumed Lanes by width where ROW < 100 and Class = A41

#fields to expose to update cursor
fields = ["SEGMENTID",       #0
          "FULLNAME",        #1
          "AB_LANE",         #2
          "BA_LANE",         #3
          "thrulanes",       #4
          "ln_exist_num",    #5
          "lane_count_exist",#6
          "lane_count_note", #7
          "mean_row_2020",   #8
          "ONEWAY",          #9
          "CLASS",           #10
          "MEDIAN_SUM_LANES",#11
          "lane_count_prop", #12
          "lane_count_rev"]  #13


count = 0
print("Aggregating lane count attributes...")
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[2] is not None and row[3] is not None):
                lanes = row[2]+row[3]   
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                
                row[7] = "Lanes values from TransCAD"
                count += 1  
                   
            elif row[4] is not None:
                
                lanes = row[4]
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "Lanes values from HPMS"
                count += 1  
                
            elif(row[5] is not None):
                lanes = row[5]
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "Lanes values from 1986 ROW Table"
                count += 1  
                
            elif(row[11] is not None):
                lanes = row[11]
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "Lanes values from DTS data"
                count += 1  
                
            elif(row[8] < 100 and row[10] == "A41"):
                lanes = 2
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "ROW <100, A41, assumed 2 lanes"
                count += 1  
                
            cursor.updateRow(row)
            
            
        except (ValueError,TypeError) as error:
            
            print("Error at SegmentID: {0} - {1},  ".format(row[0],error))
            

print("Lane counts aggregated from {} items".format(count))


### Fill null values in ROW Estimate (needs improvement)

In [ ]:
#full null ROW estimation values with assumption of 60' TO BE UPDATED AT LATER DATE

#fields to expose to update cursor
fields = ["SEGMENTID",
          "mean_row_2020",
          "row_estimate_note"]

where_clause = '"mean_row_2020" IS NULL'

count = 0
print("Filling ROW NULL values...")
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            row[1] = 60
            row[2] = "No Data. Assumed 60'."
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            
            print("Error at SegmentID: {0} - {1},  ".format(row[0],error))
            

print("ROW NULL values filled for {} items".format(count))



## Pandas - Data QC/Explorations

In [ ]:
#Create DF for CSDM analysis
df = pd.DataFrame.spatial.from_featureclass(modal_composite)

df_fields = df.columns.values.tolist()

df_fields.sort()

for i in df_fields:
     print(i)

In [ ]:
#fields = ["SEGMENTID","FULLNAME","ONEWAY","AB_VOC_MAX","BA_VOC_MAX","VOC_max","AB_LANE","BA_LANE","ln_exist_num","thrulanes","TPN_OID","NewTier","mean_row_2020","modal_width_auto_exist","modal_width_auto_prop"]

#fields = ["SEGMENTID","FULLNAME","CLASS","ONEWAY","AB_VOC_MAX","BA_VOC_MAX","VOC_max","AB_LANE","BA_LANE","ln_exist_num","thrulanes","TPN_OID","NewTier","mean_row_2020","modal_width_auto_rule","modal_width_auto_exist","modal_width_auto_prop"]

#fields = ["SEGMENTID","FULLNAME","CLASS","ONEWAY","NewTier","NBRightP_7","SBLeftPe_6","mean_row_2020","modal_width_park_exist","modal_width_ped_exist","modal_width_ped_prop","modal_width_ped_furn"]

# fields = ["CS_type","length_ft","mean_row_area","modal_width_auto_exist", "modal_width_ped_exist","modal_area_ped_E"]


# fields = ["SEGMENTID",
#           "CS_type",
#           "FULLNAME",
#           "CLASS","ONEWAY",
#           "mean_row_2020",
#           "AB_LANE", 
#           "BA_LANE", 
#           "thrulanes",
#           "ln_exist_num",
#           "MEDIAN_SUM_LANES",
#           "lane_count_exist",
#           "lane_count_note"]

fields = ["SEGMENTID",       
          "NBRightP_5",   # Pedestrian Buffer Furniture Zone (NB)
          "SBLeftPedB"]    # Pedestrian Buffer Furniture Zone (SB)

#fields = ["CS_type","length_ft","mean_row_area","modal_width_auto_exist","lane_miles"]

filt = (df["modal_width_ped_exist"]>0)

#two way streets, null lane count, row > 100
filt2 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["ONEWAY"] == 0) &
         (df["mean_row_2020"]>100))
         
#one way streets, null lane count, row > 75
filt3 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["ONEWAY"].isin([1,2])) &
         (df["mean_row_2020"]>75))

A41 = (df["CLASS"] == 'A41')

filt5 = ((~df["TPN_OID"].isnull()) &
         (df["NewTier"] == 4))


filt_lanes = (
    (df["AB_LANE"].isnull()) &
    (df["BA_LANE"].isnull()) &
    ~(df["thrulanes"].isnull()))


filt_NewTier = (
    ~(df["NewTier"].isnull()))

#misses
filt_cond_miss = (
    (df["modal_width_auto_rule"].isnull())
)
boulevard = (df["CS_type"] == 'Boulevard')

avenue = (df["CS_type"] == 'Avenue')

main_street = (df["CS_type"] == 'Main Stret')

major_street = (df["CS_type"] == 'Major Street')

street = (df["CS_type"] == 'Street')

rural = (df["CS_type"] == 'Rural Road')

lane = (df["CS_type"] == 'Lane/Alley')

HERE = (df["MEDIAN_SUM_LANES"].notnull())

pd.set_option('display.max_rows', 250)

filt_TransCAD = df["AB_LANE"].notnull()

filt_HPMS = df["thrulanes"].notnull()

lane_count = ~(df["lane_count_exist"].notnull())

row = (df["mean_row_2020"].isnull)

lane_count_note = (df["lane_count_note"] == "ROW <100, A41, assumed 2 lanes.")

# df.loc[:,fields].sort_values(by =["mean_row_2020"], ascending = False)

df.loc[:,fields]


## Modal Width Development

## Calculate Modal Widths for Auto Traffic

- Calculate existing modal with for auto traffic using lane count x average lane width into `modal_width_auto_exist`. 

- If VOC Max from TransCAD 2040 > 1.0, assume two-way streets have two additonal lanes added (22' additional width), and one-way streets have one additional lane added (11' additonal width). Proposed width is calcualted into `modal_width_auto_prop`

- If VOC Max from TransCAD 2040 is <1.0, existing width is calculted into proposed width


**Assumptions, Notes, and Parameters**
1. Average lane width parameter =  11' wide.
2. Street segments with no lane data have the following additional assumptions applied, in order of implementation: 
    1. If `ONEWAY = 0` (street is two-way) and `mean_row < 100` assume 2 lanes total for line segment. Adjust with `param_two_way`
    2. If `ONEWAY = 0` (street is two-way) and `mean_row > 100` disregard (segments are typically edge cases)
    3. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row < 75` assume 1 lane total for line segment. Adjust with `param_one_way`
    4. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row > 75` disregard (segments are typically edge cases)
3. All street segments that have lane data, but do not have TransCAD VOC Max data have `modal_width_auto_exist` calculated into `modal_width_auto_prop`
4. Remaining street segments with no lane data, no mean_row calcualtion, and no TransCAD data, but have `CLASS = A41` are assumed to be 2 lanes, 22' width (regardless of oneway/two-way)
4. All street segments that were unable to have mean_row calculated, and do not have lane data are ignored


In [ ]:
#Conditional Re-Write
#PARAMETERS
param_one_way = 75.0
param_two_way = 100.0

#fields to expose to update cursor
VOC_fields = ["SEGMENTID", #0
              "SEGMENTID", #1
              "SEGMENTID", #2
              "SEGMENTID", #3
              "SEGMENTID", #4
              "SEGMENTID", #5
              "SEGMENTID", #6
              "SEGMENTID", #7
              "SEGMENTID", #8
              "SEGMENTID", #9
              "ONEWAY",      #10
              "modal_width_auto_exist", #11
              "modal_width_auto_prop",  #12
              "AB_LANE",     #13
              "BA_LANE",     #14
              "SEGMENTID",   #15
              "ln_exist_num",#16
              "AB_VOC_MAX",  #17
              "BA_VOC_MAX",  #18 
              "mean_row_2020", #19
              "lanes_assumed", #20,
              "CLASS",         #21
              "VOC_max",    #22
              "thrulanes",   #23  HPMS Lanes
              "modal_width_auto_rule", #24,
              "lane_count_exist",   #25 - Lane Count - Existing
              "lane_count_prop"]   #26 - Lane Count - Proposed Unconstrained
             

print("Update rows for Traffic widths...")

count_01_01 = 0
count_01_02 = 0
voc_above = 0

count_02_01 = 0
count_03_01 = 0

count = 0
err_count = 0

#CONDITIONAL GROUP 01 - VOC > 1.0
where_clause = '("VOC_max" is NOT null) AND ("VOC_max" >= 1.0)'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #One Way Streets    
            if(
                (row[25] is not None) and    #Lanes
                (row[10] in [1,2])
            ):

                current_width = (row[25]*11) + 2
                expanded_width = current_width + 11              
                cond = "01_01" 
                row[11] = current_width
                row[12] = expanded_width
                row[26] = row[26] + 1     #add future lane
                row[24] = cond
                voc_above +=1 
                count_01_01 +=1
                count +=1
                
            #Two Way Streets
            elif (
                (row[25] is not None) and  #Lanes
                (row[10] == 0)         #two-way indicator
            ):
                
                current_width = (row[25]*11) + 2
                expanded_width = current_width + 22    
                cond = "01_02"
                row[11] = current_width
                row[12] = expanded_width
                row[24] = cond
                row[26] = row[26] + 2     #add future lane
                voc_above +=1
                count_01_02 +=1
                count +=1
                     
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))        
    
#CONDITIONAL GROUP 02 - VOC < 1.0 - all records calclulated the same, no lanes added        
where_clause = '("VOC_max" is NOT null) AND ("VOC_max" < 1.0)'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #All Streets
            if (row[25] is not None):
                                   
                current_width = (row[25]*11) + 2     
                cond = "02_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_02_01 +=1
                count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))            
            
#CONDITIONAL GROUP 03 - VOC NULL               
where_clause = '"VOC_max" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #All Streets
            if (row[25] is not None):
                                   
                current_width = (row[25]*11) + 2     
                cond = "03_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_03_01 +=1
                count +=1
                                                                                    
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            

print("01-01 VOC >1.0 - One Way: {}".format(count_01_01))
print("01-02 VOC >1.0 - Two Way: {}".format(count_01_02))
print("VOC Above: {}".format(voc_above))

print("02-01 VOC <1.0 - Both Ways: {}".format(count_02_01))

print("03-01 VOC NULL - Both Ways:{}".format(count_03_01))




##Add 2' width to all auto widths on streets that contain medians (additional outside lane)
median_count = 0
where_clause = 'NOT ("median_type" IS NULL) AND NOT ("modal_width_auto_rule" IS NULL)'

with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[11] is not None):    
                row[11] += 2
                row[12] += 2
                row[24] = row[24] + " MEDIAN"
                median_count += 1
        except ValueError as error:
            print("Error at SegmentID: {0} - {1}".format(row[15],error))
                     
print("median buffers calculated for {} items".format(median_count))

print("Modal Widths - Traffic complete.")

In [ ]:
df = pd.DataFrame.spatial.from_featureclass(modal_composite)

fields = ["SEGMENTID","FULLNAME","ONEWAY","VOC_max","modal_width_auto_rule","lane_count_exist"]



In [ ]:
VOC = (df["VOC_max"].isnull())
lanes = ~(df["lane_count_exist"].isnull())

modal_rule = (df["modal_width_auto_rule"].isnull())

df.loc[VOC & lanes,fields]

In [ ]:
#QC Auto Conditionals
#Create DF for CSDM analysis

#01-01 - VOC >1.0 - One Way
filt_cond_1_1 = (~(df["lane_count_exist"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         df["ONEWAY"].isin([1,2]))

#01-02 - VOC >1.0 - Two Way
filt_cond_1_2 = (~(df["lane_count_exist"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         df["ONEWAY"].isin([0]))


#02-01 - VOC <1.0 - Both Ways
filt_cond_2_1 = (~(df["lane_count_exist"].isnull()) &
         (df["VOC_max"] <1.0 ))

#03-01 - VOC NULL - Both Ways
filt_cond_3_1 = (~(df["lane_count_exist"].isnull()) &
                 (df["VOC_max"].isnull()))


print("Conditional 01_01 check..")
print(df.loc[filt_cond_1_1,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_01'),fields].shape[0])
print("\n")

print("Conditional 01_02 check..")
print(df.loc[filt_cond_1_2,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_02'),fields].shape[0])
print("\n")

###

print("Conditional 02_01 check..")
print(df.loc[filt_cond_2_1,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='02_01'),fields].shape[0])
print("\n")

###

print("Conditional 03_01 check..")
print(df.loc[filt_cond_3_1,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_01'),fields].shape[0])
print("\n")

In [ ]:
lanes = 3
lane_width = (lanes-2)*10 + 22

expanded_width = (lanes-1)*10 + 22

print(lane_width)
print(expanded_width)

## Calculate Modal Widths for Bike Facilities


- Existing width is calculated into `modal_width_bike_exist` and `modal_width_bike_prop`

- Proposed width is then updated into `modal_width_bike_prop` based on proposed and redevelopment bike plan attributes.

- If a give segment has a value for existing bike facilites, but no values for proposed facilites, existing width is calculted into proposed width.


**Assumptions, Notes, and Parameters**
1. "Protected Bike Lane", "Buffered Bike Lane" = 10'
2. "Bike Lane", "Climbing Lane", "Shoulder Bikeway" = 7'


In [ ]:
#fields to expose to update cursor for bike facilities
fields = ["Fac_Type_BE",            #0
          "modal_width_bike_exist", #1
          "Fac_Type_BP",            #2
          "Fac_Type_BR",            #3
          "modal_width_bike_prop"]  #4

#update bike existing width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite,fields,'NOT ("Fac_Type_BE" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike existing calculations
            if row[0] == "Protected Bike Lane" or row[0] == "Buffered Bike Lane":
                row[1] = 12
                row[4] = 12
                    
            elif (row[0] == "Bike Lane" or
                  row[0] == "Climbing Lane" or
                  row[0] == "Shoulder Bikeway" or
                  row[0] == "Shared Use Path" or
                  row[0] == "Shared Roadway"):
                row[1] = 7
                row[4] = 7
                    
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike existing width calculated\n")

#perform second pass w/ update cursor for bike proposed features (will over write previous calcs if there is an improvement)
#update bike proposed width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite,fields,'NOT ("Fac_Type_BP" IS NULL AND "Fac_Type_BR" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike proposed & redev calculations
            if (row[2] == "Protected Bike Lane" or
                  row[2] == "Buffered Bike Lane" or
                  row[3] == "Protected Bike Lane" or
                  row[3] == "Buffered Bike Lane"):
                row[4] = 12
                
            elif (row[2] == "Bike Lane" or
                  row[2] == "Climbing Lane" or
                  row[2] == "Shoulder Bikeway" or
                  row[2] == "Shared Use Path" or
                  row[2] == "Shared Roadway" or
                  row[3] == "Bike Lane" or
                  row[3] == "Climbing Lane" or
                  row[3] == "Shoulder Bikeway" or
                  row[3] == "Shared Use Path" or
                  row[3] == "Shared Roadway"):
                row[4] = 7
                
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike proposed width calculated\n")
print("Modal Widths - Bike complete.")

## Calculate Modal Widths for Medians

- Median data was manually classified by reviewing street segments where `mean_row_2020 > 75`, then estimaing median type and median width using aerial imagery.
- Width estimates were then mapped into `modal_width_medians` (see assumptions below)

**Assumptions, Notes, and Parameters**
1. Median width esitmations were mapped to modal with values as follows:

|Median Width Estimate| Modal Width Value  |
|:--|:--|
| < 5' |5'  |
| 5' - 10' |11' (increase by 1' to match avg. lane width)  |
| 15' - 20' |17'  |
| >20' |22'  |


In [ ]:
#Calculate Modal Width for Medians

# "A","median_width"] = 5
# "B","median_width"] = 11
# "C","median_width"] = 17
# "D","median_width"] = 22

#fields to expose to update cursor
fields = ["median_type",               #0
          "median_width",              #1
          "modal_width_medians_exist", #2
          "modal_width_medians_prop"]  #3

#update median width columns
print("Update rows for Medians widths..")
count = 0
with arcpy.da.UpdateCursor(modal_composite,fields,'NOT ("median_type" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        
        try:
            if row[1] == "A":
                median_width = 7
                row[2] = median_width
                row[3] = median_width
                count +=1
            elif row[1] == "B":
                median_width = 13
                row[2] = median_width
                row[3] = median_width
                count +=1
            elif row[1] == "C":
                median_width = 19
                row[2] = median_width
                row[3] = median_width
                count +=1
            elif row[1] == "D":
                median_width = 14
                row[2] = median_width
                row[3] = median_width
                count +=1
                    
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("median widths calculated for {} items".format(count))
print("Modal Widths - Medians complete.")

## Calculate Modal Widths for on-street Parking segments

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
fields = arcpy.ListFields(modal_composite)

for field in fields:
    if field.name == "modal_width_ped_exist":
        print(field.type)

In [ ]:
#calculate modal width for street segements with on-street parking facilities


#expose fields to update cursor
parking_fields = ["SEGMENTID",              #0 - SegmentID
                  "NBRightP_7",             #1 - Road Buffer (NB)
                  "SBLeftPe_6",             #2 - Road Buffer (SB)
                  "NBRightP_4",             #3 - Ped Intrusion (NB)
                  "SBLeftPe_4",             #4 - Ped Intrusion (SB)
                  "modal_width_park_exist", #5 - Modal Width - Park (Existing)
                  "modal_width_park_prop",  #6 - Modal Width - Park (Proposed)
                  "mean_row_2020",          #7 - Mean ROW Analysis
                  "modal_width_auto_exist", #8 - Modal Width - Auto (Existing)
                  "CLASS"]                  #9 - Street Class (tiger)

park_ped_count = 0
park_assume_count = 0
no_can_park_count = 0
err_count = 0

param = 22

print("Update rows for Parking widths..")

#calculate parking width for segments based on ped inventory facilities for NB road side
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"NBRightP_7" IS NOT NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[1] == 1) or (row[3] == 1)):
                
                row[5] = 8
                row[6] = 8
                
               
            elif(row[1] == 0 and row[3] == 0):
                row[5] = 0
                row[6] = 0  
                
            cursor.updateRow(row)
            park_ped_count += 1
            
        except (ValueError,TypeError) as error:  
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

            
#calculate parking width for segments based on ped inventory facilities for SB road side               
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"SBLeftPe_6" IS NOT NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[2] == 1) or (row[4] == 1)):
                row[5] = row[5]+8
                row[6] = row[6]+8
                
            cursor.updateRow(row)
            park_ped_count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
               

#calculate parking width for segments not listed in ped inventory facilities
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"NBRightP_7" IS NULL AND "SBLeftPe_6" IS NULL AND "CLASS" = \'A41\'') as cursor:
    for row in cursor:
        #print("row")
        try:
            if (row[7] is not None) and (row[8] is not None) and ((row[7] - row[8]) > param):
                row[5] = 16
                row[6] = 16
                park_assume_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
            
#calculate parking width to 0 for all other segments
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"modal_width_park_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            row[5] = 0
            row[6] = 0
            no_can_park_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Parking widths (ped) calculated for {0} items".format(park_ped_count))
print("Parking widths (assumed) calculated for {0} items".format(park_assume_count))
print("Parking widths (0) calculated for {0} items".format(no_can_park_count))

print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Parking complete.")

## Calculate Modal Widths for Ped Facilities

- Existing width is calculated into `modal_width_ped_exist` and `modal_width_ped_prop`

- Proposed width `modal_width_ped_prop` is then updated if a segment had no ped facilites, was not listed in ped facilites, or was listed in "Candidate Walkways" or "Candidate Upgrades", and the proposed width was greater than the existing width. (see assumptions below)


**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Any segment listed in `NewTier` == 4 is set to 8' if oneway, 16' if two-way for future widths.
3. The following steet segments types are set to 12' for `modal_width_ped_prop`
    1. Any street segment listed in "Candidate Walkways", and "Candidate Upgrades" datasets
    2. Any street segment listed in Pedestrian Sidewalk Inventory as having no facilites, but are not listed in "Candidate Walkways" or "Candidate Upgrades"
    3. Any street segment NOT listed in Pedestrian Sidewalk Inventory, AND not listed in "Candidate Walkways", AND not listed in "Candidate Upgrades", with `CLASS = A41`


In [ ]:
#add field to hold furniture zone width

#store field, field alias in dict
new_fields = {
              "modal_width_ped_furn": "Modal Width - Ped (Furn)"
}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_composite,field_name=key,field_type="SHORT", field_alias = value)
    
print("Field additions complete")

### Setup

In [ ]:
#calculate modal width for ped facilities

#assumptions based on ped data dictionary values for PED ZONE WIDTH
# greater than 10' = 12' DOMAIN 04
# 5'-10' (clear and obstruced types) = 8' DOMAIN 03, DOMAIN 02
# less than 5' = 4' DOMAIN 01

#Ped Zone Widths
p1 = 12 #DOMAIN VALUE 4 -      Greater than 10
p2 = 8  #DOMAIN VALUE 3 and 2 - 5' to 10'
p3 = 4  #DOMAIN VALUE 1

#existing ped facilities

#expose fields to update cursor
ped_fields = ["SEGMENTID",                #0 - SegmentID
                "NBRightP_1",             #1 - Pedestrian Zone Width (NB)
                "SBLeftPe_1",             #2 - Pedestiran Zone Width (SB)
                "modal_width_ped_exist",  #3 - Modal Width - Ped (Existing)
                "modal_width_ped_prop",   #4 - Modal Width - Ped (Proposed)
                "mean_row_2020",          #5 - Mean ROW Analysis
                "FID_Ped_Improve_temp",   #6 - Ped Improvements
                "FID_Ped_Add_temp",       #7 - Ped Additions
                "CLASS",                  #8 - Street Class (tiger)
                "NewTier",                #9 - Bus Transit - New Proposed Tier
                "ONEWAY",                 #10 - Oneway indicator
                "NBRightP_5",             #11 - Pedestrian Buffer Furniture Zone (NB)
                "SBLeftPedB",             #12 - Pedestrian Buffer Furniture Zone (SB)
                "Fac_Type_BE",            #13 - Bike - Existing Type
                "Fac_Type_BP",            #14 - Bike - Proposed Type
                "Fac_Type_BR",            #15 - Bike - ReDev Type
                "modal_width_park_exist", #16 - Modal Width - Park (Existing)
                "modal_width_park_prop",  #17 - Modal Width - Park (Proposed)
                "modal_width_ped_furn",   #18 - Modal Width - Ped (Furniture Zone)
                "modal_width_bus_prop"]   #19 - Modal Width - Bus (Existing)
     


### Calculate Ped Zone Widths

In [ ]:

count = 0
err_count = 0

print("Update rows with Ped Zone widths..")

print("Calculate widths for NB Side...")
# where_clause = 'NOT ("NBRightP_1" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields) as cursor:
    for row in cursor:
        try:
            if row[1] == 4:
                width = 12
                row[3] = width
                row[4] = width       
            
            elif (row[1] in [2,3]):
                width = 8
                row[3] = width
                row[4] = width

            elif (row[1] == 1):
                width = 4
                row[3] = width
                row[4] = 4

            elif (row[1] == 0 and row[2] == 0):
                width = 0
                row[3] = width
                row[4] = 12
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Calculate widths for SB Side")
where_clause = 'NOT ("SBLeftPe_1" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        try:
            if row[2] == 4:
                row[3] += 12
                row[4] += 12      
            
            elif (row[2] in [2,3]):
                row[3] += 8
                row[4] += 8

            elif (row[2] == 1):
                row[3] += 4
                row[4] += 4
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
        
print("Calculate widths for NULLS")
where_clause = '("NBRightP_1" IS NULL) AND ("SBLeftPe_1" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        try:
            
            row[3] = 0
            row[4] = 12      
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))

### Calculate & Add Furniture Zone Widths

In [ ]:
# Add furniture zone widths
print("Update rows by adding Ped Furniture widths..")

#Ped Furniture Widths
f1 = 6  #DOMAIN VALUE 3 - Above 4'
f2 = 4  #DOMAIN VALUE 2 - 0' to 4'
f3 = 0  #DOMAIN VALUE 1 - 0

count = 0
err_count = 0
where_clause = 'NOT ("NBRightP_5" IS NULL OR "SBLeftPedB" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            #12' total
            if row[11] == 3 and row[12] == 3:
            
                row[3] += 12
                row[4] += 12
                row[18] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
        
            #10' total
            elif ((row[11] == 3 and row[12] == 2)or
                  (row[11] == 2 and row[12] == 3)):
                
                row[3] += 10
                row[4] += 10
                row[18] = 10
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #8' total
            elif (row[11] == 2 and row[12] == 2):
         
                row[3] += 8
                row[4] += 8
                row[18] = 8
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #4' total
            elif ((row[11] == 2 and row[12] == 1)or
                  (row[11] == 1 and row[12] == 2)):
        
                row[3] += 4
                row[4] += 4
                row[18] = 4
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
           #0' fill for all others
            elif ((row[11] is None) and
                  (row[12] is None)):
        
                row[18] = 0
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                  
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))
            
#with arcpy.da.UpdateCursor(modal_composite_05a, ped_e_fields,'"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL AND "CLASS" = \'A41\' ') as cursor:



### Add additional ped space for Bus Stops from Bus Transit NewTiers

In [ ]:
print("Updating rows for NewTier values...")         
            
#proposed bus transit New tiers
count = 0
err_count = 0

where_clause = 'NOT ("NewTier" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:        
        try:
            #01 - Tier 3, 4 - Two Way Streets - if ped furn + parking < 20, add difference to ped
            if (
                (row[10] is not None) and
                (row[10] == 0) and         #OneWay
                (row[9] in [3,4]) and      #NewTier
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 20)    #Parking + Ped Furn
                ):
                
                cond = 1
                diff = (20-(row[16]+row[18]))
                row[4] = row[4]+diff
                
            #02 - Tier 2 - Two Way Strets + Protected Bike - if ped furn + parking < 20, add difference to ped   
            elif (
                (row[10] is not None) and
                (row[10] == 0) and           #OneWay
                (row[9] in [2]) and          #NewTier
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 20) and  #Parking + Ped Furn
                ((row[13] == "Protected Bike Lane") or (row[14] == "Protected Bike Lane") or (row[15] == "Protected Bike Lane"))
                ):
                            
                cond =2
                diff = (20-(row[16]+row[18]))
                row[4] = row[4]+diff
            
            #03 - Tier 3, 4 - One Way Streets - if ped furn + parking < 10, add difference to ped
            elif(
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[9] in [3,4]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) <10)
                ):
                
                cond=3
                diff = (10-(row[16]+row[18]))
                row[4] = row[4]+diff
                           
                           
            #04 - Tier 2 - One Way Streets + Protected Bike - if ped furn + parking < 10, add difference to ped   
            elif (
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[9] in [2]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 10) and
                ((row[13] == "Protected Bike Lane") or (row[14] == "Protected Bike Lane") or (row[15] == "Protected Bike Lane"))
                ):
                
                cond=4
                diff = (10-(row[16]+row[18]))
                row[4] = row[4]+diff
                           
            #05 - Tier 1 or (Tier 2 & Not Protected Bike) - Two Way Streets - if ped furn + parking < 16, add difference to ped
            elif (
                (row[10] is not None) and
                (row[10] in [0]) and
                (row[16] is not None) and
                (row[18] is not None) and
                (int((row[16]) + int(row[18])) < 16) and
                ((row[9] == 1) or (row[9] == 2 and not
                        ((row[13] == "Protected Bike Lane") or
                        (row[14] == "Protected Bike Lane") or
                        (row[15] == "Protected Bike Lane"))))
                ):
                
                cond=5
                sum_val = row[16]+row[18]
                diff = (16-(int(row[16])+int(row[18])))
                
                print("SegmentID: {0}, Parking: {1}, Furn: {2}, Total: {3}, Adding: {4}".format(row[0],row[16],row[18],sum_val, diff))
                print(type(row[4]))
                print(type(diff))
                row[4] = row[4] + int(diff)
            
            #06 - Tier 1 or (Tier 2 & Not Protected Bike) - One Way Streets - if ped furn + parking < 8, add difference to ped
            elif (
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) <8) and
                ((row[9] == 1) or (row[9] == 2 and not
                        ((row[13] == "Protected Bike Lane") or
                        (row[14] == "Protected Bike Lane") or
                        (row[15] == "Protected Bike Lane"))))
                ):
                
                cond = 6
                diff = (8-(row[16]+row[18]))
                row[4] = row[4]+diff
                                                 
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0}, Cond:{1}, Parking:{2}, Furniture:{3} - {4} ".format(row[0],cond,row[16], row[18], error))
                
   

### Proposed Ped - increase space if current is inadeaquate

In [ ]:
#proposed ped facilities - use a pass with update cursor to update proposed widths to 12' if they
# have segments listed in Ped improvement datasets, and the existing width is less than 12'
count = 0
err_count = 0

where_clause = 'NOT ("FID_Ped_Improve_temp" IS NULL OR "FID_Ped_Add_temp" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_e_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #only update proposed width if is less than the current width
            if ((row[4] is not None) and row[4] < 12):
                row[4] = 12
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            



### 0' and No Data - set future width to min of 12'

In [ ]:
# perform final pass with update cursor to add proposed width to facilites that have no sidewalks
# and are not covered by sidewalk inventory, but are residential (class = 'A41')
count = 0
err_count = 0

where_clause = '"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, ped_e_fields, where_clause) as cursor:
    for row in cursor:
        #print("final pass")
        try:
            #only update proposed width if is less than the current width
            if (row[8] == 'A41' and ((row[4] is not None and row[4] < 12) or row[4] is None)):
            #if ((row[8] == 'A41') and (row[4] is None)):
                row[4] = 12   
                #print("pass 3 for A41")
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

print("Modal Widths - Ped complete.") 

## Calculate Modal Widths for Bus Transit segments

TEXT TO BE REPLACED

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
#calculate modal width for bus facilities


#expose fields to update cursor
bus_fields = ["SEGMENTID",              #0 - SegmentID
              "NAME",                   #1 - Name
              "modal_width_bus_exist",  #2 - Modal Width - Bus (Existing)
              "modal_width_bus_prop",   #3 - Modal Width - Bus (Proposed)
              "mean_row_2020",          #4 - Mean ROW Analysis
              "CLASS",                  #5 - Street Class (tiger)
              "NewTier",                #6 - Bus Transit - New Tier
              "ONEWAY"]                 #7 - OneWay

bus_count = 0
new_count = 0
err_count = 0

#existing bus facilites by SegmentID (replace with Bus Plan data when available)

IDs_exist = (
    18813,18858,18867,18914,18968,18984,19024,19033,19068,19090,19140,19148,19148,19234,19272,
    19306,19320,19333,19357,19401,19441,19501,19595,19672,19758,19790,21993,22030,22049,22112,
    22163,22169,33146)

where_clause = '"SEGMENTID" IN {}'.format(IDs_exist)

print("Update rows for Bus widths..")
print("Calculating existing...")
#calculate existing Priority Bus facilities
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"SEGMENTID" IN {}'.format(IDs_exist)) as cursor:   
    for row in cursor:
        #print("row")
        try:
            if row[1] == "HOTEL":
                row[2] = 24
                row[3] = 24
                bus_count += 1
            
            elif row[1] == "KALAKAUA" or row[1] == "KING":
                row[2] = 12
                row[3] = 12
                bus_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Existing calc done.")
            
#calculate proposed Tier4 Bus facilities
print("Calculating proposed...")
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"NewTier" = 4') as cursor:
    for row in cursor:
        try:
            if ((row[7] is not None) and row[7] == 0):
                row[3] = 24
                new_count += 1
            elif((row[7] is not None) and row[7] in [1,2]):
                row[3] = 12
                new_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
print("Proposed calc done.") 
            
print("Bus widths - Existing -  calculated for {0} segments".format(bus_count))
print("Bus widths - New -       calculated for {0} segments".format(new_count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Bus complete.")

## Calculate Unconstrained Metrics

Use previously calculated modal widths for each modal type to generate a total unconstrained modal width value.


**Assumptions, Notes, and Parameters**
1. Future Tier 4 Bus Priority lanes are ignored in the unconstrained width calcuation based on the assumption that Bus Priority lanes will always be placed in existing auto lanes.
2. Do not sum Ped Width and Bike Width if future Bike Type = Shared Use Path
3. Ignore all bike facility types of shared road way in calculation



### Unconstrained Modal Width

In [ ]:
#calculate unconstrained modal widths

fields = ["SEGMENTID",               #0
          "modal_width_ped_prop",    #1
          "modal_width_bike_prop",   #2
          "modal_width_auto_prop",   #3
          "modal_width_bus_prop",    #4
          "modal_width_park_prop",   #5
          "modal_width_medians",     #6
          "modal_width_max",         #7
          "Fac_Type_BP",             #8
          "Fac_Type_BR",             #9
          "NewTier"]                  #10        

count = 0
err_count = 0

#desc

print("Calculating width..")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #igonore shared road-way widths from bike width
            if row[8] == "Shared Roadway" or row[9] == "Shared Roadway":     
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #ignore bike shared use path for ped > 7    
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] > 7:
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] < 7:
                row[7] = sum(filter(None,(row[1], 7,  row[3], row[4], row[5], row[6])))
                count +=1
            
            #summarize all other segments
            else:
                row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
                count +=1
                
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[5], row[6])))
            
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Unconstrained widths calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Unconstrained complete.")


In [ ]:
#add additonal fields for modal width/area metrics

#length_ft - to capture line length in ft prior to conversion to WebMercator. Calculated from shape_length
#mean_row_area - estimated area, 2020 row estimate X length
#modal_area_max - estimated unconstrained area, unconstrained width x length
#modal_area_diff - difference between Unconstrained Area and 2020 Estimate. Uncon - 2020. Positive values will represent exceedences, negative will be surplus 


print("adding fields")
arcpy.AddField_management(modal_composite_05a,field_name="length_ft",field_alias = "Length (ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="mean_row_area",field_alias = "ROW Area Estimate (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_max",field_alias = "Unconstrained Modal Area (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_diff",field_alias = "Unconstrained Modal Area Exceedence (sq. ft.)",field_type="FLOAT")
print("done")

In [ ]:
print("adding fields")
arcpy.AddField_management(modal_composite,field_name="modal_area_auto_E",field_alias = "Modal Area - Auto - Existing",field_type="FLOAT")
arcpy.AddField_management(modal_composite,field_name="modal_area_auto_U",field_alias = "Modal Area - Auto - Unconstrained",field_type="FLOAT")
arcpy.AddField_management(modal_composite,field_name="modal_area_auto_C",field_alias = "Modal Area - Auto - Constrained",field_type="FLOAT")

arcpy.AddField_management(modal_composite,field_name="modal_area_ped_E",field_alias = "Modal Area - Ped - Existing",field_type="FLOAT")
arcpy.AddField_management(modal_composite,field_name="modal_area_ped_U",field_alias = "Modal Area - Ped - Unconstrained",field_type="FLOAT")
arcpy.AddField_management(modal_composite,field_name="modal_area_ped_C",field_alias = "Modal Area - Ped - Constrained",field_type="FLOAT")

arcpy.AddField_management(modal_composite,field_name="modal_area_bike_E",field_alias = "Modal Area - Bike - Existing",field_type="FLOAT")
arcpy.AddField_management(modal_composite,field_name="modal_area_bike_U",field_alias = "Modal Area - Bike - Unconstrained",field_type="FLOAT")
arcpy.AddField_management(modal_composite,field_name="modal_area_bike_C",field_alias = "Modal Area - Bike - Constrained",field_type="FLOAT")

print("done")

### Unconstrained Modal Areas

In [ ]:
#calculate additional modal metrics

fields = ["SEGMENTID",       #0
          "modal_width_max", #1
          "Shape_Length",    #2
          "length_ft",       #3
          "mean_row_area",   #4
          "modal_area_max",  #5
          "modal_area_diff", #6
          "mean_row_2020"]   #7
   

count = 0
err_count = 0

#desc

print("Calculating areas..")

with arcpy.da.UpdateCursor(modal_composite, fields, 'NOT "mean_row_2020" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            length_ft = row[2]
            mean_row_area = length_ft * row[7]
            modal_area_max = length_ft * row[1]
            modal_area_diff = modal_area_max - mean_row_area
            
            row[3] = length_ft
            row[4] = mean_row_area
            row[5] = modal_area_max
            row[6] = modal_area_diff
            
            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))

print("Modal Areas - complete.")

In [ ]:
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

field_check = "length_ft"



fields = ["SEGMENTID",               #0
          "length_ft",               #1
          "modal_width_auto_exist",  #2
          "modal_width_auto_prop",   #3
          "modal_width_auto_const",   #4
          "modal_width_ped_exist",   #5
          "modal_width_ped_prop",   #6
          "modal_width_ped_const",   #7
          "modal_width_bike_exist", #8
          "modal_width_bike_prop",  #9
          "modal_width_bike_const", #10
          "modal_area_auto_E",#11
          "modal_area_auto_U",#12
          "modal_area_auto_C",#13
          "modal_area_ped_E",#14
          "modal_area_ped_U",#15
          "modal_area_ped_C",#16
          "modal_area_bike_E",#17
          "modal_area_bike_U",#18
          "modal_area_bike_C"]#19  

for field in fields:
    if field in field_names:
        print("{} field alreay exists".format(field))
        
    else:
        print("not found")
    

In [ ]:
#calculate modal area fields 2


new_fields = {
              "lane_count": "Transit Lanes - Existing",
              "lane_count_prop": "Transit Lanes - Proposed (Unconstrained)",
              "lane_count_rev": "Transit Lanes - Proposed (Constrained)",
              "lane_miles": "Transit Lane Miles - Existing",
              "lane_miles_prop": "Transit Lane Miles - Proposed (Unconstrained)",
              "lane_miles_rev": "Transit Lane Miles - Proposed (Constrained)",
              "lane_count_note": "Transit Lanes - Note"
}

fields = ["SEGMENTID",               #0
          "length_ft",               #1
          "modal_width_auto_exist",  #2
          "modal_width_auto_prop",   #3
          "modal_width_auto_const",   #4
          
          "modal_width_ped_exist",   #5
          "modal_width_ped_prop",   #6
          "modal_width_ped_const",   #7
          
          "modal_width_bike_exist", #8
          "modal_width_bike_prop",  #9
          "modal_width_bike_const", #10
          
          "modal_area_auto_E",#11
          "modal_area_auto_U",#12
          "modal_area_auto_C",#13
          
          "modal_area_ped_E",#14
          "modal_area_ped_U",#15
          "modal_area_ped_C",#16
          
          "modal_area_bike_E",#17
          "modal_area_bike_U",#18
          "modal_area_bike_C",#19
          "lane_miles",       #20
          "lane_miles_prop",  #21
          "lane_count",       #22
          "lane_count_prop"]  #23

count = 0
err_count = 0

#desc

print("Calculating modal areas..")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[2] is not None) and
               (row[3] is not None)):
                
                row[11] = row[2]*row[1] #Modal Area - Auto Existing
                row[12] = row[3]*row[1] #Modal Area - Auto Proposed (Unconstrained)

            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))


with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[14] is not None) and
               (row[15] is not None)):
            
                row[14] = row[5]*row[1] #Modal Area - Pedestrian Existing
                row[15] = row[6]*row[1] #Modal Area - Pedestrian Proposed (Unconstrained)

            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))



with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[17] is not None) and
               (row[18] is not None)):
                
                row[17] = row[8]*row[1] #Modal Area - Bike Existing
                row[18] = row[9]*row[1] #Modal Area - Bike Proposed (Unconstrained)

            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[22] is not None) and
               (row[23] is not None)):
            
                row[20] = (row[22]*row[1])/5280   #Lane Miles (existing)
                row[21] = (row[23]*row[1])/5280

            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))


print("Modal Areas 2 - complete.")

In [ ]:
#reset all (or some) width and area fields to null (use during development)

fields = ["modal_width_ped_exist",     #0
          "modal_width_ped_prop",      #1
          "modal_width_bike_exist",     #2
          "modal_width_bike_prop",     #3
          "modal_width_auto_exist",     #4
          "modal_width_auto_prop",     #5
          "modal_width_bus_exist",    #6
          "modal_width_bus_prop",     #7
          "modal_width_park_exist",  #8
          "modal_width_park_prop",  #9
          "modal_width_medians_exist",   #10
          "modal_width_medians_prop",   #11
          "modal_width_max",       #12
          "mean_row_area",         #13
          "modal_area_max",         #14
          "modal_area_diff"]         #15

err_count = 0

print("Reseting modal widths and areas to Null...")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            
                      
#             row[0] = None    # modal_width_ped_exist
#             row[1] = None    # modal_width_ped_prop
#             row[2] = None    # modal_width_bike_exist
#             row[3] = None    # modal_width_bike_prop
#             row[4] = None    # modal_width_auto_exist
#             row[5] = None    # modal_width_auto_prop
#             row[6] = None    # modal_width_bus_exist
#             row[7] = None    # modal_width_bus_prop
#             row[8] = None    # modal_width_park_exist
#             row[9] = None    # modal_width_park_prop
            row[10] = None   # modal_width_medians_exist
            row[11] = None   # modal_width_medians_prop
#             row[12] = None   # modal_width_max

            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print(error)

print("All widths and areas reset to Null.")
            